In [7]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold , cross_val_score
from sklearn import metrics 
from matplotlib import pyplot as plt
from numpy import mean
from numpy import absolute
import openpyxl
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, accuracy_score, plot_roc_curve, plot_confusion_matrix, roc_curve, confusion_matrix
from matplotlib import pyplot
from tensorflow.keras.models import load_model
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score, plot_roc_curve, plot_precision_recall_curve, plot_confusion_matrix

In [8]:
# Import feature list
infile = open('../04_predictionUserHistory/01_data/FINALsmallSampleSet_3months_without_duplicates.pkl','rb')
import_file = pickle.load(infile)
infile.close()
df = import_file
feature_cols = list(df.columns)

In [9]:
def get_metrics(clf,X,y):
    pred = clf.predict(X)
    a = accuracy_score(y,pred)
    p = precision_score(y,pred)
    r = recall_score(y,pred)
    roc_auc = roc_auc_score(y,pred)
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    fpr = fp/(fp+tn)

    return a,p,r,roc_auc,fpr

In [10]:
# list subgroups
fair_metrics = pd.DataFrame(columns=['model', 'group', 'subgroup', 'Accuracy', 'Precision', 'Recall', 'AUC', 'FPR'])
matrice = ['double_df_abi','double_df_keinAbi','double_df_boys','double_df_girls','double_df_deutsch','double_df_migration','double_df_buch0','double_df_buch1']
group = ['abiEltern', 'abiEltern', 'gender', 'gender', 'erstsprache', 'erstsprache', 'buecher', 'buecher']
subgroup = ['abi', 'keinAbi', 'boys', 'girls', 'deutsch', 'migration', 'buch0', 'buch1']

In [ ]:
# DECISION TREE
DTE_model = pickle.load(open('../04_predictionUserHistory/02_decisionTreeNoGender/DecisionTreemodel_3months.pkl', 'rb'))

for (group, subgroup, matrice) in zip(group, subgroup, matrice):
    print(matrice)
    path= matrice+'.pkl'
    infile = open(path,'rb')
    df = pickle.load(infile)
    infile.close()

    dataset= df[df.columns[df.columns.isin(feature_cols)]]
    #X_new = X.loc[~X.set_index(list(X.columns)).index.isin(trainX.set_index(list(trainX.columns)).index)]

    y = dataset['Erfolg']
    X = dataset.drop(columns=['Erfolg','Sex__m','Sex__w'])
    a,p,r,roc_auc,fpr = get_metrics(DTE_model,X,y)
    fair_metrics = fair_metrics.append({'model':'DTE','group':group,'subgroup':subgroup,'Accuracy':a,'Precision': p, 'Recall':r, 'AUC':roc_auc, 'FPR':fpr}, ignore_index=True)

In [12]:
df = fair_metrics

In [13]:
grouped = df.groupby(df.group)
# df_abiEltern = grouped.get_group("abiEltern")
df_gender = grouped.get_group("gender")
# df_erstsprache = grouped.get_group("erstsprache")
# df_buecher = grouped.get_group("buecher")

In [15]:
df_gender = df_gender.drop(columns=['group', 'Accuracy'])#'Unnamed: 0',
df_gender = pd.pivot_table(df_gender, values=["Precision","Recall","AUC","FPR"], index=["model"], columns=["subgroup"])
df_gender['PP'] = df_gender.Precision.girls-df_gender.Precision.boys
df_gender['EO'] = df_gender.Recall.boys-df_gender.Recall.girls
df_gender['SA'] = df_gender.AUC.girls-df_gender.AUC.boys
df_gender['PE'] = df_gender.FPR.boys-df_gender.FPR.girls

# df_erstsprache = df_erstsprache.drop(columns=['group', 'Unnamed: 0', 'Accuracy'])
# df_erstsprache = pd.pivot_table(df_erstsprache, values=["Precision","Recall","AUC","FPR"], index=["model"], columns=["subgroup"])
# df_erstsprache['PP'] = df_erstsprache.Precision.deutsch-df_erstsprache.Precision.migration
# df_erstsprache['EO'] = df_erstsprache.Recall.migration-df_erstsprache.Recall.deutsch
# df_erstsprache['SA'] = df_erstsprache.AUC.deutsch-df_erstsprache.AUC.migration
# df_erstsprache['PE'] = df_erstsprache.FPR.migration-df_erstsprache.FPR.deutsch


# df_buecher = df_buecher.drop(columns=['group', 'Unnamed: 0', 'Accuracy'])
# df_buecher = pd.pivot_table(df_buecher, values=["Precision","Recall","AUC","FPR"], index=["model"], columns=["subgroup"])
# df_buecher['PP'] = df_buecher.Precision.buch1-df_buecher.Precision.buch0
# df_buecher['EO'] = df_buecher.Recall.buch0-df_buecher.Recall.buch1
# df_buecher['SA'] = df_buecher.AUC.buch1-df_buecher.AUC.buch0
# df_buecher['PE'] = df_buecher.FPR.buch0-df_buecher.FPR.buch1

# df_abiEltern = df_abiEltern.drop(columns=['group', 'Unnamed: 0', 'Accuracy'])
# df_abiEltern = pd.pivot_table(df_abiEltern, values=["Precision","Recall","AUC","FPR"], index=["model"], columns=["subgroup"])
# df_abiEltern['PP'] = df_abiEltern.Precision.abi-df_abiEltern.Precision.keinAbi
# df_abiEltern['EO'] = df_abiEltern.Recall.keinAbi-df_abiEltern.Recall.abi
# df_abiEltern['SA'] = df_abiEltern.AUC.abi-df_abiEltern.AUC.keinAbi
# df_abiEltern['PE'] = df_abiEltern.FPR.keinAbi-df_abiEltern.FPR.abi

#df_buecher = df_buecher.drop(columns=['AUC','Precision','Recall','FPR'])
#df_erstsprache = df_erstsprache.drop(columns=['AUC','Precision','Recall','FPR'])
df_gender = df_gender.drop(columns=['AUC','Precision','Recall','FPR'])
#df_abiEltern = df_abiEltern.drop(columns=['AUC','Precision','Recall','FPR'])

#df_buecher.columns = df_buecher.columns.droplevel(1)
#df_erstsprache.columns = df_erstsprache.columns.droplevel(1)
df_gender.columns = df_gender.columns.droplevel(1)
#df_abiEltern.columns = df_abiEltern.columns.droplevel(1)

#df_buecher = pd.pivot_table(df_buecher, values=["PP","EO","SA","PE"],  columns=["model"])
df_gender = pd.pivot_table(df_gender, values=["PP","EO","SA","PE"], columns=["model"])
#df_erstsprache = pd.pivot_table(df_erstsprache, values=["PP","EO","SA","PE"], columns=["model"])
#df_abiEltern = pd.pivot_table(df_abiEltern, values=["PP","EO","SA","PE"], columns=["model"])

In [16]:
#format result

def threshold001(v, props=''):
    return props if (v > 0.02) or (v < -0.02) else None

def threshold005(v, props=''):
    return props if (v > 0.05) or (v < -0.05) else None

def negativeValue(v, props=''):
    return props if (v < 0) else None

def showTable(df):
    styled = df.style.set_properties(color="black", align="right")\
        .set_properties(**{'background-color': 'white'})\
        .applymap(threshold001, props='color:orange;')\
        .applymap(threshold005, props='color:red;')\
        .applymap(negativeValue, props='font-weight:bold;')
    return styled

In [17]:
s = showTable(df_gender)
s

model,DTE
EO,0.001797
PE,-0.001389
PP,-0.000037
SA,-0.001593
